In [2]:
import carla
import math
import time
import queue
import numpy as np
import cv2

In [3]:
client=carla.Client('localhost',2000)
world=client.load_world('Town02')

In [4]:
world

In [5]:
bp_lib=world.get_blueprint_library()

In [6]:
vehicle_bp=bp_lib.find("vehicle.tesla.model3")#在蓝图库中找tesla的模型

In [7]:
import random
spawn_points = world.get_map().get_spawn_points()
#get_map获取虚拟世界的地图信息，包含拓扑结构、道路网络和建筑物位置
#spawn生成点，初始化虚拟世界
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
#随机选择生成点生成车辆
# spawn camera
camera_bp = bp_lib.find('sensor.camera.rgb')
#用这个蓝图来创建一个具有 RGB 摄像头传感器的虚拟世界实体
camera_init_trans = carla.Transform(carla.Location(z=2))
#在z=2m的高度处生成了一个摄像头实体
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)
#将摄像头附着到了车辆的实体上
vehicle.set_autopilot(True)

# Set up the simulator in synchronous mode
settings = world.get_settings()
settings.synchronous_mode = True 
#同步模式
settings.fixed_delta_seconds = 0.05
#每隔0.05秒进行一步仿真，每秒20帧图片渲染
world.apply_settings(settings)

# 生成点对象
spawn_points = world.get_map().get_spawn_points()

# 传感器的图片输入到一个image队列之中
image_queue = queue.Queue()
camera.listen(image_queue.put)

In [8]:
#这个函数通常用于计算相机的投影矩阵，
#用于将三维世界坐标映射到二维图像坐标，以便进行摄像头校准、3D到2D的投影和其他计算机视觉或计算机图形任务。
def build_projection_matrix(w, h, fov):
    focal = w / (2.0 * np.tan(fov * np.pi / 360.0))
    K = np.identity(3)
    K[0, 0] = K[1, 1] = focal
    K[0, 2] = w / 2.0
    K[1, 2] = h / 2.0
    return K

In [9]:
#目的是将三维世界坐标映射到摄像头图像的二维坐标上，用于计算图像中的点
def get_image_point(loc, K, w2c):
        # Calculate 2D projection of 3D coordinate

        # Format the input coordinate (loc is a carla.Position object)
        point = np.array([loc.x, loc.y, loc.z, 1])
        # transform to camera coordinates
        point_camera = np.dot(w2c, point)

        # New we must change from UE4's coordinate system to an "standard"
        # (x, y ,z) -> (y, -z, x)
        # and we remove the fourth componebonent also
        point_camera = [point_camera[1], -point_camera[2], point_camera[0]]

        # now project 3D->2D using the camera matrix
        point_img = np.dot(K, point_camera)
        # normalize
        point_img[0] /= point_img[2]
        point_img[1] /= point_img[2]

        return point_img[0:2]

In [10]:
world_2_camera = np.array(camera.get_transform().get_inverse_matrix())
#反转变换矩阵可以用来将虚拟世界中的坐标从相机坐标系转换回世界坐标系。
# Get the attributes from the camera
image_w = camera_bp.get_attribute("image_size_x").as_int()#图片的宽
image_h = camera_bp.get_attribute("image_size_y").as_int()#图片的高
fov = camera_bp.get_attribute("fov").as_float()#该属性表示摄像头的视场角（Field of View）

# Calculate the camera projection matrix to project from 3D -> 2D
K = build_projection_matrix(image_w, image_h, fov)

In [11]:
bounding_box_set = world.get_level_bbs(carla.CityObjectLabel.TrafficLight)
bounding_box_set.extend(world.get_level_bbs(carla.CityObjectLabel.TrafficSigns))

# Remember the edge pairs
edges = [[0,1], [1,3], [3,2], [2,0], [0,4], [4,5], [5,1], [5,7], [7,6], [6,4], [6,2], [7,3]]


In [12]:
for i in range(50):
    vehicle_bp = random.choice(bp_lib.filter('vehicle'))
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
    if npc:
        npc.set_autopilot(True)

In [13]:
while True:
    # Retrieve and reshape the image
    world.tick()
    image = image_queue.get()

    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

    # Get the camera matrix 
    world_2_camera = np.array(camera.get_transform().get_inverse_matrix())


    for npc in world.get_actors().filter('*vehicle*'):

        # Filter out the ego vehicle
        if npc.id != vehicle.id:

            bb = npc.bounding_box#非自己车辆，其他的所有实体
            dist = npc.get_transform().location.distance(vehicle.get_transform().location)

            # Filter for the vehicles within 50m
            if dist < 50:

            #计算了车辆的前向向量和连接两辆车的向量。
            # 如果点积的结果大于1，这表示这两个向量之间的夹角小于90度
            #并且它们的方向是大致一致的。在这个上下文中，这意味着另一辆车辆位于摄像头前方，
            #车辆的前向方向与连接向量的方向大致一致。因此，通过这个条件判断，代码只会绘制位于摄像头前方的车辆的边界框。
                forward_vec = vehicle.get_transform().get_forward_vector()
                ray = npc.get_transform().location - vehicle.get_transform().location

                if forward_vec.dot(ray) > 1:
                    p1 = get_image_point(bb.location, K, world_2_camera)
                    verts = [v for v in bb.get_world_vertices(npc.get_transform())]
                    for edge in edges:
                        p1 = get_image_point(verts[edge[0]], K, world_2_camera)
                        p2 = get_image_point(verts[edge[1]],  K, world_2_camera)
                        cv2.line(img, (int(p1[0]),int(p1[1])), (int(p2[0]),int(p2[1])), (255,0,0, 255), 1)        

    cv2.imshow('ImageWindowName',img)
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()

: 